This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
#%env NASDAQ_API_KEY=''

In [2]:
!  pip3 install python-dotenv


In [3]:

# get api key from your .env file
import os
from dotenv import load_dotenv

#load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

None


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
# Import package
import requests
from bs4 import BeautifulSoup
import json


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-01api_key=API_KEY'
r=requests.get(url)
json_data=r.json()
print(json.dumps(json_data,indent=2))

{
  "dataset": {
    "id": 10095370,
    "dataset_code": "AFX_X",
    "database_code": "FSE",
    "name": "Carl Zeiss Meditec (AFX_X)",
    "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
    "refreshed_at": "2020-12-01T14:48:09.907Z",
    "newest_available_date": "2020-12-01",
    "oldest_available_date": "2000-06-07",
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "frequency": "daily",
    "type": "Time Series",
    "premium": false,
    "limit": null,
    "transform": null,
    "column_index": null,
    "start_date": "2017-01-01",
    "end_date": "2017-01-01",
    "data": [],
    "collapse": null,
    "order": null,
    "database_id": 6129
  }
}


#  1 & 2Collecting Data for the year 2017 and converting into dictionary

In [6]:
#Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
#for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# Assign URL to variable: url
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31api_key=API_KEY'

# Package the request, send the request and catch the response: r
r=requests.get(url)

# Decode the JSON data into a dictionary: json_data
#Convert the returned JSON object into a Python dictionary.
json_data17=r.json()

In [7]:
type(json_data17)

dict

In [9]:
json_data17.keys()

dict_keys(['dataset'])

In [10]:
json_data17['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [11]:
json_data17['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [12]:
json_data17['dataset']['data'][0:4]


[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None],
 ['2017-12-27',
  51.45,
  51.89,
  50.76,
  51.82,
  None,
  57452.0,
  2957018.0,
  None,
  None,
  None],
 ['2017-12-22',
  51.05,
  51.5,
  50.92,
  51.32,
  None,
  71165.0,
  3641949.0,
  None,
  None,
  None]]

In [13]:
json_data17A=json_data17['dataset']['data']

In [14]:
type(json_data17A)

list

# Calculate what the highest and lowest opening prices were for the stock in this period.

In [15]:
opening_price=[]
for i in json_data17A:
    if i[1]!=None:
        opening_price.append(i[1])
print(opening_price)

[51.76, 51.65, 51.45, 51.05, 51.16, 51.88, 52.73, 52.37, 52.7, 53.11, 52.64, 52.29, 52.28, 51.5, 50.89, 50.8, 51.21, 49.5, 49.52, 48.64, 49.64, 49.09, 49.13, 49.11, 48.8, 48.4, 47.25, 46.57, 47.03, 47.09, 47.98, 48.4, 48.38, 47.3, 47.65, 46.42, 46.16, 45.81, 45.0, 45.88, 46.29, 46.53, 45.48, 45.2, 45.01, 45.16, 44.9, 45.08, 45.72, 46.01, 45.8, 45.61, 45.5, 45.58, 45.97, 45.64, 46.2, 46.19, 46.01, 45.36, 44.51, 43.58, 42.0, 42.35, 42.3, 42.3, 41.48, 42.29, 42.54, 42.65, 42.5, 42.29, 42.35, 42.49, 43.21, 42.81, 42.7, 43.0, 42.66, 43.0, 42.38, 42.16, 42.0, 42.0, 41.71, 42.11, 42.64, 42.72, 42.82, 42.46, 42.42, 42.28, 41.88, 42.4, 42.53, 42.12, 41.3, 41.73, 43.5, 44.9, 45.85, 45.13, 45.34, 45.25, 45.24, 44.94, 45.26, 45.16, 44.91, 44.7, 45.31, 45.57, 45.74, 45.06, 45.5, 45.6, 45.07, 44.67, 44.29, 44.94, 44.64, 44.79, 45.5, 44.67, 45.83, 45.29, 45.01, 45.73, 46.68, 47.23, 46.95, 47.29, 47.03, 47.46, 46.48, 46.9, 45.66, 46.34, 46.52, 46.5, 47.31, 46.77, 47.8, 47.01, 47.12, 46.8, 46.12, 45.22

In [16]:
max_opening= max(opening_price)
min_opening=min(opening_price)

print("The highest opening price is:", max_opening , "The lowest opening prices is:", min_opening)

The highest opening price is: 53.11 The lowest opening prices is: 34.0


# What was the largest change in any one day (based on High and Low price)?

In [17]:
diff_price=[]
for i in json_data17A:
    if i[1]!=None and i[2]!=None:
        diff=i[2]-i[3]
        diff_price.append(diff)
highest_change = round(max(diff_price), 2)
print("The maximum difference in highest and lowest price is:", highest_change)

The maximum difference in highest and lowest price is: 2.81


# What was the largest change between any two days (based on Closing Price)?

In [18]:
closeprice=[]
prevprice=[]
for i in json_data17A:
    if i[4]!=None:
        closeprice.append(i[4])
for j in json_data17A[1:]:
    if j[4]!=None:
        prevprice.append(j[4])
price_diff=[]
zip_object=zip(closeprice,prevprice)
for closeprice_i,prevprice_i in zip_object:
    price_diff.append(abs(closeprice_i-prevprice_i))
print("The maximum change in one day:",round(max(price_diff),2))



   


    


The maximum change in one day: 2.56


# What was the average daily trading volume during this year?

In [19]:
daily_vol=[]
for i in json_data17A:
    if i[6]!=None:
        daily_vol.append(i[6])
avg_daily_vol=round((sum(daily_vol)/len(daily_vol)),2)
print("The average daily trading volume is :",avg_daily_vol)
        

The average daily trading volume is : 89124.34


# What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [21]:
def median(lst):
    n = len(lst)
    s = sorted(lst)
    return (s[n//2-1]/2.0+s[n//2]/2.0, s[n//2])[n % 2] if n else None
print(" The median daily volume is:",median(daily_vol))
#sorted_daily_vol=sorted(daily_vol)
#print(sorted_daily_vol[0:15])
#median_daily_vol=sorted_daily_vol[(round(len(sorted_daily_vol)/2))]
#print(len(sorted_daily_vol))
#print(median_daily_vol)
#print(len(sorted_daily_vol))
# Median volume is the 255

 The median daily volume is: 76286.0


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)